In [18]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [19]:
#Test Code| Version 2#
Year='1938'
Showa='13'
#Read Motherdata
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\"+Year+"\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

In [20]:
#Test code| Version 2#
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

PositionList=['Manager','Leader','Admin','Outsource','Engineer1','Engineer2']

In [21]:
# Function for returning x coordinate of Position name
def Detect_Position(Json,Position):
    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue
            
    ### Get Coordinates ###
    if Position=='Manager':        
        res = [d
           for d in Dict 
           if ('主事' in d['inferText']) or ('主' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Managers on this page!")
            return(Edge)
        else:
            print("No Managers on this page!")
            return([])
    
    if Position=='Leader':        
        res = [d
           for d in Dict 
           if ('課長' in d['inferText']) or ('課' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Leader on this page!")
            return(Edge)
        else:
            print("No Leader on this page!")
            return([])
        
    if Position=='Engineer1':        
        res = [d
           for d in Dict 
           if ('技師' in d['inferText']) or ('師' in d['inferText'])]
        print(res)
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Engineer1 on this page!")
            return(Edge)
        else:
            print("No Engineer1 on this page!")
            return([])
    
    if Position=='Engineer2':        
        res = [d
           for d in Dict 
           if ('技手' in d['inferText']) or ('手' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Engineer2 on this page!")
            return(Edge)
        else:
            print("No Engineer2 on this page!")
            return([])
        
    if Position=='Admin':        
        res = [d
           for d in Dict 
           if ('事務員' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Admin on this page!")
            return(Edge)
        else:
            print("No Admin on this page!")
            return([])
        
    if Position=='Outsource':
        res = [d
           for d in Dict 
           if '嘱託員' in d['inferText']]
        print(res)
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            print("Found Outsource on this page!")
            return(Edge)
        else:
            print("No Engineer on this page!")
            return([])
        
    if Position=='Clerk':
        res = [d
           for d in Dict 
           if d['inferText']=='書記']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            print("No Clerk on this page!")
            return([])
        
    if Position=='Worker':
        res = [d
           for d in Dict 
           if d['inferText']=='雇']
        print(res)
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            print("No Worker on this page!")
            return([])
        
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Dept,Office,Unit,ImagePage):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Unit+"\\"
    with open(path+"Image"+str(ImagePage)+".png",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [37]:
#Test code| Version 2#
Fail=0
for n in range(0,len(df)):
    #Show Working office#
    #Extract key info of office
    if n==len(df):
        print('FINISHED!')
        break
    Row  = df.iloc[n]
    NxRow= df.iloc[n+1]
    

    ###Collect Location information###
    Page=int(Row["Page"])
    NxPage=int(NxRow["Page"])

    Dept=Row["Dept"]
    NxDept=NxRow["Dept"]

    Office=Row["Office"]
    NxOffice=NxRow["Office"]
    
    try:
        UnitList=dta[Year][Dept][Office]['Units']['List']
        print(UnitList)
    except:
        Fail=Fail+1
        continue
    
    print(Dept,Office)
    for Unit in UnitList:
        #Convert to json via CLOVA
        JsonList=[]
        for l in range(1,len(dta[str(Year)][Dept][Office]['Units'][Unit]['Page_Range'])+1):
            try:
                Json=Clova(Dept,Office,Unit,l)                
            except FileNotFoundError:
                break
            JsonList.append(Json)
            Index=0
            for element in JsonList:
                if 'fields' not in list(element.keys()):
                    JsonList=JsonList[-Index]
                Index=Index+1
        
            #Find Starting X coordinates of Positions.
            PositionListNow=PositionList.copy()
            try:
                Page_Range=dta[str(Year)][Dept][Office][Unit]['Page_Range']
            except:
                Fail=Fail+1
                continue
                
            CordDict={}
            Page=Page_Range[0]

            if len(JsonList)==7:
                continue
            else:
                for k in range(0,len(JsonList)):
                    CordList=list(map(Detect_Position,[JsonList[k]]*len(PositionListNow),PositionListNow))
                    Positions= {PositionListNow[i]:CordList[i] for i in range(len(PositionListNow))}
                    Positions = {k:v for k,v in Positions.items() if v}    
                    print(Positions)

                    PosiList = list(Positions.keys())
                    CordList = list(Positions.values())

                    CordDictNow= {PosiList[i]: {'XLocation':CordList[i],'Page':Page} for i in range(len(PosiList))}
                    CordDictNow = {k:v for k,v in CordDictNow.items() if v}
                    CordDict.update(CordDictNow)
                    Page=Page+1

                    for x in list(CordDictNow.keys()):
                      PositionListNow.remove(x)

                    if len(PositionListNow)==0:
                        break
                dta[Year][Dept][Office][Unit]['Position']=CordDict

            ##Add Ending information
            Frame=pd.DataFrame.from_dict(dta[Year][Dept][Office][Unit]['Position'],orient='index')
            if len(Frame)==0:
                continue
            Frame=Frame.sort_values(by = ['Page', 'XLocation'], ascending = [False, True])
            if len(dta[Year][Dept][Office]['Position'])==0:
                continue
            Frame["EndLocation"]=list(Frame['XLocation'].shift(-1))[:-1]+list([dta[Year][Dept][Office][Unit]["Office_X2"]])
            Frame["EndPage"]=list(Frame['Page'].shift(-1))[:-1]+list([dta[Year][NxDept][NxOffice]["Starting_Page"]])
            Frame=Frame.to_dict('index')
            dta[Year][Dept][Office][Unit]['Position']=Frame
            print(FailList)

['議事掛']
Admin 秘書課
['庶務掛', '人事掛', '給與掛', '福利掛', '打算九下掛']
Admin 人事課
['庶務掛', '法規掛', '文書卦掛', '公報掛', '圖書掛']
Admin 文書課
['庶務掛', '議事掛']
企画局 庶務課
['管理掛', '豫算掛', '調査掛']
企画局 財務課
['庶務掛', '會計掛', '收入掛', '支拂掛', '簿記掛']
経理局 会計課
['庶務掛', '公債掛', '臨時佛貨公債訴訟掛']
経理局 公債課
['庶務掛', '第一契約掛', '第二契約掛', '用品掛', '檢査掛']
経理局 用度課
['庶務掛', '第一用地掛', '第一市有地掛', '第二市有地掛', '管理掛', '測量掛', '抜師掛', '査定掛']
経理局 地理課
['Mid勸業掛', '金融掛', '小賈市場掛']
産業局 商工課
['農務掛', '水産掛', '畜産掛', '負債整理掛']
産業局 農漁課
['指導掛', '取締掛']
産業局 権度課
['庶務掛', '主計掛']
教育局 庶務課
['勞務掛', '職業掛', '授 掛', '輔導掛']
社会局 職業課
['庶務掛', '監理掛', '經理掛', '技師(參考所醫務課醫務掛']
保健局 庶務課
['業務掛', '收納掛', '給水工事掛']
水道局 業務課
['工務掛']
水道局 給水課
['庶務掛', '福利掛', '教育搗掛']
電気局 労働課
['檢收掛', '契約掛', '營繕掛']
電気局 経理課
['管財掛', '出納掛', '倉庫掛', '主事だよ掛']
電気局 会計課
['庶務掛', '計畫掛', '給與掛']
運輸部 庶務課
['電車掛', '自動車掛', '計算掛', '事故掛', '主事李迷惑香烟山重助掛']
運輸部 業務課
['乘客掛', '宣傳掛', '主事年一七〇〇生中学村保掛', '廣告掛']
運輸部 事業普及課
['庶務掛', '營業掛', '普及掛', '收納掛']
電燈部 営業課
['管理掛', '外線掛']
電燈部 配線課
['庶務掛', '第一監察掛', '第二監察掛', '第三監察掛']
監査部 監察部
['事務掛', '防務掛', '計畫掛']
市民動員部 防衛課
['統制掛', '計畫掛']


IndexError: single positional indexer is out-of-bounds

In [41]:
Fail/len(df)

2.0833333333333335

In [38]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+str(Year)+'\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)

In [39]:
dta

{'1938': {'Admin': {'秘書課': {'Starting_Page': 2,
    'Office_X1': 480,
    'Ending_Page': 2,
    'Office_X2': 178,
    'Page_Range': [2],
    'Units': {'議事掛': {'Page': 1,
      'StartLocation': 245,
      'EndPage': 1,
      'EndLocation': 0,
      'Page_Range': [1]},
     'List': ['議事掛']}},
   '人事課': {'Starting_Page': 2,
    'Office_X1': 168,
    'Ending_Page': 4,
    'Office_X2': 377,
    'Page_Range': [2, 3, 4],
    'Units': {'庶務掛': {'Page': 1,
      'StartLocation': 100,
      'EndPage': 2,
      'EndLocation': 402,
      'Page_Range': [1, 2]},
     '人事掛': {'Page': 2,
      'StartLocation': 402,
      'EndPage': 2,
      'EndLocation': 267,
      'Page_Range': [2]},
     '給與掛': {'Page': 2,
      'StartLocation': 267,
      'EndPage': 2,
      'EndLocation': 132,
      'Page_Range': [2]},
     '福利掛': {'Page': 2,
      'StartLocation': 132,
      'EndPage': 2,
      'EndLocation': 112,
      'Page_Range': [2]},
     '打算九下掛': {'Page': 2,
      'StartLocation': 112,
      'EndPage': 2,
